In [1]:
import numpy as np
import matplotlib.pyplot as plt
import casadi
import scipy.interpolate
from autoware_vehicle_adaptor.param import parameters

# Set Parameters

In [ ]:
# constraint
max_velocity = 11.1 # 11m/s=40km/h # max velocity limit [m/s]
max_lateral_accel = 0.8 # max lateral acceleration limit [m/ss] # https://github.com/autowarefoundation/autoware.universe/blob/98c24b824c68605c28513de47a36aaf5a4a89817/planning/motion_velocity_smoother/config/default_motion_velocity_smoother.param.yaml#L15
min_acc = -1.0
max_acc = 1.0
min_jerk = -1.0
max_jerk = 1.0
stopping_velocity = 2.778 # https://github.com/autowarefoundation/autoware.universe/blob/82a6c0c5d0e3a7a3803f92c9a1334047b80344c5/planning/motion_velocity_smoother/config/default_motion_velocity_smoother.param.yaml#L34C5-L34C29

max_steering_angle_rate = 40.0 # maximum steering angle rate [degree/s] # https://github.com/autowarefoundation/autoware.universe/blob/98c24b824c68605c28513de47a36aaf5a4a89817/planning/motion_velocity_smoother/config/default_motion_velocity_smoother.param.yaml#L22C5-L22C86

# vehicle parameter
wheel_base = parameters.wheel_base

# path parameter
dpos = 0.01
dsteer_dpos_max = 0.04

straight_line_length = 300.0 # first straight line len
terminal_straight_line_length = 300.0
terminal_straight_line_velocity = 10.0

ra = 20.0
rb = 40.0
rc = 60.0
ra_th_end = np.pi/6.0
rb_th_end = np.arccos( ra/rb * (np.cos(ra_th_end)-1.0) + 1.0 )
rc_th_end = np.arccos( rb/rc * (np.cos(rb_th_end)-1.0) + 1.0 )
print("ra_th_end", ra_th_end, "rb_th_end", rb_th_end, "rc_th_end", rc_th_end)

ra_steer = np.arctan(wheel_base/ra) # [dyaw = dpos * tan(steer) / wheel_base] and [dpos = r*dyaw]
rb_steer = np.arctan(wheel_base/rb)
rc_steer = np.arctan(wheel_base/rc)

print("ra_steer", ra_steer, "rb_steer", rb_steer, "rc_steer", rc_steer)

In [3]:
def get_curve_path(initial_path_point, yaw_target, steer_target):

    x = initial_path_point[0]
    y = initial_path_point[1]
    yaw = initial_path_point[2]
    steer = initial_path_point[3]
    
    temp_path=[]    
    
    while True:
        x += dpos*np.cos(yaw)
        y += dpos*np.sin(yaw)
        yaw += dpos * np.tan(steer) / wheel_base
        steer += np.clip( steer_target-steer, a_min=-dsteer_dpos_max*dpos, a_max=dsteer_dpos_max*dpos )
        temp_path.append([x,y,yaw,steer])
        if yaw>yaw_target:
            break
        if len(temp_path)>50000:
            break

    
    while True:
        if len(temp_path)>30000:
            break
        x += dpos*np.cos(yaw)
        y += dpos*np.sin(yaw)
        yaw += dpos * np.tan(steer) / wheel_base
        steer += np.clip( -steer_target-steer, a_min=-dsteer_dpos_max*dpos, a_max=dsteer_dpos_max*dpos )
        temp_path.append([x,y,yaw,steer])
        if yaw<-yaw_target:
            break

    while True:
        if len(temp_path)>30000:
            break
        x += dpos*np.cos(yaw)
        y += dpos*np.sin(yaw)
        yaw += dpos * np.tan(steer) / wheel_base
        steer += np.clip( steer_target-steer, a_min=-dsteer_dpos_max*dpos, a_max=dsteer_dpos_max*dpos )
        temp_path.append([x,y,yaw,steer])
        if yaw>=0:
            break

    
    return np.array(temp_path)

def get_straight_line_path(initial_path_point,path_len, gain):
    x = initial_path_point[0]
    y = initial_path_point[1]
    yaw = initial_path_point[2]
    steer = initial_path_point[3]
    
    temp_path=[]
    while len(temp_path) < path_len/dpos:
        x += dpos*np.cos(yaw)
        y += dpos*np.sin(yaw)
        yaw += dpos * np.tan(steer) / wheel_base        
        steer += np.clip( - gain * yaw - steer , a_min=-dsteer_dpos_max*dpos, a_max=dsteer_dpos_max*dpos )
        temp_path.append([x,y,yaw,steer])
    return np.array(temp_path)

    

In [ ]:
total_path = np.array([ [i*dpos,0,0,0] for i in range(int(straight_line_length/dpos))])
total_path = np.vstack([total_path,get_curve_path(initial_path_point=total_path[-1], yaw_target=ra_th_end, steer_target=ra_steer)])
total_path = np.vstack([total_path,get_curve_path(initial_path_point=total_path[-1], yaw_target=rb_th_end, steer_target=rb_steer)])
total_path = np.vstack([total_path,get_curve_path(initial_path_point=total_path[-1], yaw_target=rc_th_end, steer_target=rc_steer)])
slalom_complete_idx = len(total_path)
total_path = np.vstack([total_path,get_straight_line_path(initial_path_point=total_path[-1], path_len=terminal_straight_line_length, gain=1.0)])
print(total_path.shape)

In [ ]:

plt.figure(figsize=(12, 6))
plt.plot(total_path[:,0],total_path[:,1])
plt.xlabel("position x",fontsize=20)
plt.ylabel("position y",fontsize=20)
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.ylim([-5,10])
# plt.savefig("path_pos.png")
plt.show()

# plt.figure(figsize=(12, 6))
plt.plot(total_path[:,0],total_path[:,2])
plt.xlabel("position x",fontsize=20)
plt.ylabel("yaw",fontsize=20)
plt.show()

# plt.figure(figsize=(12, 6))
plt.plot(total_path[:,0],total_path[:,3])
plt.xlabel("position x",fontsize=20)
plt.ylabel("steer",fontsize=20)
plt.show()



In [6]:
# Initial velocity limit

In [ ]:
velocity_limit = np.array([max_velocity]*len(total_path))
velocity_limit[slalom_complete_idx:] *= terminal_straight_line_velocity/max_velocity
plt.plot(total_path[:,0], velocity_limit,label="external velocity limit")
plt.xlabel("position x",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.ylim([-1,max_velocity+1])
plt.legend()

# Applying stopping velocity limit at straight line path

In [ ]:
applied_velocity_limit = []
for i in range(len(total_path)):
    if (0.3*straight_line_length)<total_path[i,0] and (0.4*straight_line_length)>total_path[i,0]:
        applied_velocity_limit.append(stopping_velocity)
    else:
        applied_velocity_limit.append(np.inf)
applied_velocity_limit = np.array(applied_velocity_limit)
plt.plot(total_path[:,0],velocity_limit, label="current velocity limit")
plt.plot(total_path[:,0],applied_velocity_limit, label="stopping velocity limit")
plt.legend()
plt.xlabel("position x",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.ylim([-1,max_velocity+1])
plt.show()
velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Applying lateral acceleration limit
$$
    max\_lateral\_accel \ge r \left(\frac{dyaw}{dt}\right)^2 = \frac{v^2}{r}
$$
$$
    \sqrt{r\times max\_lateral\_accel} \ge v
$$

In [ ]:
#r_trajectory = wheel_base/np.abs(np.arctan(total_path[:,3])+1e-16)
r_trajectory = wheel_base/np.abs(np.tan(total_path[:,3])+1e-16)
r_trajectory_ = wheel_base/np.abs(np.arctan(total_path[:,3])+1e-16)
applied_velocity_limit = np.sqrt(max_lateral_accel*r_trajectory )

plt.plot(total_path[:,0],velocity_limit, label="current velocity limit")
plt.plot(total_path[:,0],applied_velocity_limit, label="lateral acceleration limit")

plt.legend()
plt.xlabel("position x",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.ylim([-1,max_velocity+1])

velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Applying steering rate limit
$$
    steer\_rate\_limit \ge \left|\frac{dsteer}{dt}\right| = \left|\frac{dpos}{dt} \frac{dsteer}{dpos}\right| = v\left|\frac{dsteer}{dpos}\right|
$$
$$
    \frac{steer\_rate\_limit}{\left|\frac{dsteer}{dpos}\right|} \ge v
$$

In [ ]:
dsteer_dpos = (total_path[1:,3]-total_path[:-1,3])/dpos
dsteer_dpos = np.hstack([dsteer_dpos,dsteer_dpos[-1]])
applied_velocity_limit = (max_steering_angle_rate*np.pi/180.0) / np.abs(1e-16+dsteer_dpos)
print(applied_velocity_limit.shape)
plt.plot(total_path[:,0],velocity_limit, label="current velocity limit")
plt.plot(total_path[:,0],applied_velocity_limit, label="steering rate limit")
plt.ylim([-1,max_velocity+1])
plt.xlabel("position x",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.legend()

velocity_limit = np.stack([velocity_limit,applied_velocity_limit]).min(axis=0)

# Final velocity limit

In [ ]:
plt.plot(total_path[:,0],velocity_limit, label="final velocity limit")
plt.ylim([-1,max_velocity+1])
plt.xlabel("position x",fontsize=15)
plt.ylabel("longitudinal velocity",fontsize=15)
plt.legend()

# velocity smoothing optimization (simplified version)
$$
    \begin{matrix}
    \min & -vel^2 + \lambda_1 * (d vel/ d pos)^2 + \lambda_2 * (d^2 vel/ d pos^2)^2 \\
    \mbox{subject to } & 0 \le vel \le vel_{limit} \\ 
    & acc_{min}\le acc \le acc_{max} \\
    & jerk_{min}\le jerk \le jerk_{max}
    \end{matrix}
$$

In [ ]:
lambda_1 = 1e-8
lambda_2 = 1e-8
n =len(velocity_limit)
v = casadi.SX.sym('v', n)
w = [v]
lbw = [0]*n
ubw = velocity_limit.tolist()


g = []
lbg = []
ubg = []
J = - (v.T @ v)
print("check 0")
for i in range(n-1):
    a = 0.5 * (v[i+1] * v[i+1] - v[i] * v[i])/dpos
    J += lambda_1 * (v[i+1] - v[i]) * (v[i+1] - v[i]) / (dpos * dpos)
    g.append(a)
    lbg.append(min_acc)
    ubg.append(max_acc)
for i in range(n-2):
    #a1 = 0.5 * (v[i+1] * v[i+1]- v[i] * v[i])/dpos    
    #a2 = 0.5 * (v[i+2] * v[i+2] - v[i+1] * v[i+1])/dpos
    jerk = 0.25 * (v[i+2] * v[i+2] - 2 * v[i+1] * v[i+1] + v[i] * v[i]) * (v[i] + v[i+1]) / (dpos * dpos)
    J += lambda_2 * (v[i+2] - 2 * v[i+1] + v[i]) * (v[i+2] - 2 * v[i+1] + v[i]) / (dpos * dpos * dpos * dpos)
    g.append(jerk)
    lbg.append(min_jerk)
    ubg.append(max_jerk)

print("check 1")
nlp = {'f': J, 'x': casadi.vertcat(*w), 'g': casadi.vertcat(*g)} 
print("check 2")
solver = casadi.nlpsol('S','ipopt',nlp)
print("check 3")
sol = solver(x0=([0]*n), lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)

# Smoothed velocity

In [ ]:
v_sol = sol["x"].toarray().flatten()
plt.title("smoothed velocity plots")
plt.plot(dpos * np.arange(total_path.shape[0]),velocity_limit, label="final velocity limit")
plt.plot(dpos * np.arange(total_path.shape[0]),v_sol, label="smoothing result")
plt.ylim([-1,max_velocity+1])
plt.xlabel("travel distance [m]")
plt.ylabel("longitudinal velocity [m/s]")
plt.show()

In [25]:
# Total path before interpolation

In [ ]:
dt_path = 2 * dpos/(v_sol[1:] + v_sol[:-1])
timestamp = []
xy_path = []
v_path = []
yaw_path = []
steer_path = []
t = 0.0
for i in range(len(dt_path)):
    timestamp.append(1.0*t)
    xy_path.append(total_path[i,[0,1]])
    v_path.append(v_sol[i])
    yaw_path.append(total_path[i,2])
    steer_path.append(total_path[i,-1])
    t += dt_path[i]
timestamp = np.array(timestamp)
xy_path = np.array(xy_path)
v_path = np.array(v_path)
yaw_path = np.array(yaw_path)
steer_path = np.array(steer_path)

a_path = (v_path[1:] - v_path[:-1]) / (timestamp[1:] - timestamp[:-1])
a_path = np.hstack([a_path, a_path[-1]])
jerk_path = (a_path[1:] - a_path[:-1]) / (timestamp[1:] - timestamp[:-1])
jerk_path = np.hstack([jerk_path, jerk_path[-1]])
dsteer_path = (steer_path[1:] - steer_path[:-1]) / (timestamp[1:] - timestamp[:-1])
dsteer_path = np.hstack([dsteer_path,dsteer_path[-1]])

trajectory_data=np.zeros((timestamp.shape[0],9))
trajectory_data[:,0] = timestamp
trajectory_data[:,[1,2]] = xy_path 
trajectory_data[:,3] = v_path
trajectory_data[:,4] = yaw_path
trajectory_data[:,5] = a_path
trajectory_data[:,6] = steer_path
trajectory_data[:,7] = jerk_path
trajectory_data[:,8] = dsteer_path
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(18, 5), tight_layout=True)
fig.suptitle("total path before interpolation")
title_name_list = ["x", "y", "vel", "yaw", "acc", "steer", "jerk", "dsteer"]
unit_name_list = ["[m]", "[m]", "[m/s]", "[rad]", "[m/s^2]", "[rad]", "[m/s^3]", "[rad/s]"]
for i in range(8):
    axes[int(i/4),i%4].plot(trajectory_data[:,0],trajectory_data[:,i+1])
    axes[int(i/4),i%4].set_title(title_name_list[i] + "_path")
    axes[int(i/4),i%4].set_xlabel("Time [s]")
    axes[int(i/4),i%4].set_ylabel(title_name_list[i] + " " + unit_name_list[i])
plt.show()

In [ ]:
timestamp_interp = []
diff_step_for_jerk = 20
min_dt = 0.05
for i in range(timestamp.shape[0]):
    timestamp_interp.append(timestamp[i])
    if i == timestamp.shape[0] - 1:
        continue
    else:
        while timestamp_interp[-1] + min_dt < timestamp[i+1]:
            timestamp_interp.append(timestamp_interp[-1] + min_dt)
timestamp_interp = np.array(timestamp_interp)
trajectory_interpolator_list = [
            scipy.interpolate.interp1d(trajectory_data[:, 0], trajectory_data[:, 1 + i])
            for i in range(trajectory_data.shape[1] - 1)
        ]
trajectory_interp = np.zeros((timestamp_interp.shape[0],9))
trajectory_interp[:,0] = timestamp_interp
for i in range(8):
    trajectory_interp[:,i+1] = trajectory_interpolator_list[i](timestamp_interp)
trajectory_interp[:-1,5] = (trajectory_interp[1:,3] - trajectory_interp[:-1,3])/(timestamp_interp[1:] - timestamp_interp[:-1])
trajectory_interp[-1,5] = trajectory_interp[-2,5]
trajectory_interp[diff_step_for_jerk:-diff_step_for_jerk,7] = (trajectory_interp[2 * diff_step_for_jerk:,5] - trajectory_interp[:- 2 * diff_step_for_jerk,5])/(timestamp_interp[2 * diff_step_for_jerk:] - timestamp_interp[:- 2 * diff_step_for_jerk])
trajectory_interp[:-1,8] = (trajectory_interp[1:,6] - trajectory_interp[:-1,6])/(timestamp_interp[1:] - timestamp_interp[:-1])
trajectory_interp[-1,8] = trajectory_interp[-2,6]

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(18, 5), tight_layout=True)
fig.suptitle("total path after interpolation")
for i in range(8):
    axes[int(i/4),i%4].plot(trajectory_interp[:,0],trajectory_interp[:,i+1])
    axes[int(i/4),i%4].set_title(title_name_list[i] + "_path")
    axes[int(i/4),i%4].set_xlabel("Time [s]")
    axes[int(i/4),i%4].set_ylabel(title_name_list[i] + " " + unit_name_list[i])
plt.show()

In [28]:
np.savetxt("slalom_course_data.csv",trajectory_interp,delimiter=',')